In [4]:
import torch
import numpy as np

In [42]:
class Network():

    num_RX_ant = None
    num_TX_ant = None
    num_scatterers = None
    lam = None
    Ant_dist = None
    TX_locs = None
    RX_locs = None
    SC_locs = None
    rand_ph = None

class pair():
    def __init__(self, N, RX_idx, TX_idx):
        self.TX_idx = TX_idx
        self.RX_idx = RX_idx
        self.num_RX_ant = N.num_RX_ant
        self.num_TX_ant = N.num_TX_ant
        self.num_scatterers = N.num_scatterers
        self.lam = N.lam
        self.Ant_dist = N.Ant_dist
        self.TX_locs = N.TX_locs
        self.RX_locs = N.RX_locs
        self.SC_locs = N.SC_locs
        self.rand_ph = N.rand_ph
        self.T_locs = [[i*self.Ant_dist+self.TX_locs[self.TX_idx][0], self.TX_locs[self.TX_idx][1]]  for i in range(self.num_TX_ant)]
        self.R_locs = [[i*self.Ant_dist+self.RX_locs[self.RX_idx][0], self.RX_locs[self.RX_idx][1]]  for i in range(self.num_RX_ant)]
        self.S_locs = [i for i in torch.from_numpy(self.SC_locs[self.RX_idx, self.TX_idx])]

    def calculate_Bt(self):
        Bt = torch.zeros(self.num_TX_ant, self.num_scatterers[self.RX_idx, self.TX_idx]+1, dtype=torch.complex64)
        # qT = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])
        qT = (self.RX_locs[self.RX_idx] - self.TX_locs[self.TX_idx])/torch.norm((self.RX_locs[self.RX_idx] - self.TX_locs[self.TX_idx]))
        D = torch.tensor(self.T_locs) - torch.tile(self.TX_locs[self.TX_idx], (self.num_TX_ant, 1))
        Bt[:, 0] = torch.exp(((-2*torch.pi*1j)/self.lam)*(torch.matmul(qT, torch.transpose(D, 0, 1))))
        for i, S_loc in enumerate(self.S_locs):
            # qT = (S_loc - TX_locs[self.TX_idx])
            qT = (S_loc - self.TX_locs[self.TX_idx])/torch.norm((S_loc - self.TX_locs[self.TX_idx]))
            Bt[:, i+1] = torch.exp(((-2*torch.pi*1j)/self.lam)*(torch.matmul(qT, torch.transpose(D, 0, 1))))
        return Bt

    def calculate_Br(self):
        Br = torch.zeros(self.num_RX_ant, self.num_scatterers[self.RX_idx, self.TX_idx]+1, dtype=torch.complex64)
        # qR = (RX_locs[self.RX_idx] - TX_locs[self.TX_idx])
        qR = (self.RX_locs[self.RX_idx] - self.TX_locs[self.TX_idx])/torch.norm((self.RX_locs[self.RX_idx] - self.TX_locs[self.TX_idx]))
        D = torch.tensor(self.R_locs) - torch.tile(self.RX_locs[self.RX_idx], (self.num_RX_ant, 1))
        Br[:, 0] = torch.exp(((2*torch.pi*1j)/self.lam)*(torch.matmul(qR, torch.transpose(D, 0, 1))))
        for i, S_loc in enumerate(self.S_locs):
            # qR = (S_loc - RX_locs[self.RX_idx])
            qR = (self.RX_locs[self.RX_idx] - S_loc)/torch.norm((S_loc - self.RX_locs[self.RX_idx]))
            Br[:, i+1] = torch.exp(((2*torch.pi*1j)/self.lam)*(torch.matmul(qR, torch.transpose(D, 0, 1))))
        return Br

    def calculate_A(self, L = None):
        A = torch.zeros(len(self.S_locs)+1, len(self.S_locs)+1, dtype=torch.complex64)
        r = torch.norm((self.RX_locs[self.RX_idx] - self.TX_locs[self.TX_idx]))
        A[0, 0] = (torch.exp(-2*torch.pi*(r/self.lam)*1j))/r
        if self.rand_ph:
            for i, S_loc in enumerate(self.S_locs):
                r = torch.norm((S_loc - self.TX_locs[self.TX_idx])) + torch.norm((self.RX_locs[self.RX_idx] - S_loc))
                random_phase = torch.rand(1) * 2 * torch.pi
                A[i+1, i+1] = ((torch.exp(-2*torch.pi*(r/self.lam)*1j))*(torch.exp(random_phase*1j)))/r
        else:
            for i, S_loc in enumerate(self.S_locs):
                r = torch.norm((S_loc - self.TX_locs[self.TX_idx])) + torch.norm((self.RX_locs[self.RX_idx] - S_loc))
                # rand_ph = torch.rand(1) * 2 * torch.pi
                # rand_ph = torch.tensor(torch.pi/6)
                phase = L[i]
                A[i+1, i+1] = ((torch.exp(-2*torch.pi*(r/self.lam)*1j))*(torch.exp(phase*1j)))/r
        return A

In [9]:
def calculate_rate(H, sigma):
    n_r = H.shape[0]
    I = torch.eye(n_r, dtype=H.dtype)
    M = I + H @ sigma @ H.conj().T
    rate = torch.log2(torch.det(M))
    return rate

In [10]:
def calculate_erg_rate(num_samples, sigma):
    erg_rate = 0
    for _ in num_samples:
        Br = p.calculate_Br()
        Bt = p.calculate_Bt()
        A = p.calculate_A()
        H = Br @ A @ Bt.conj().T
        erg_rate += calculate_rate(H, sigma)
    return erg_rate


In [11]:
def f(x):
    return 2*x

In [ ]:
x = torch.tensor(2+3j, 

In [ ]:
x = torch.tensor(2+1j, requires_grad=True)

# Define the function f(x) = x^2
f = x**3

g = f.real

# Perform backpropagation
g.backward()
.5* x.grad.conj()

tensor(6.-4.5000j)

In [61]:
x = torch.tensor(2+1j, requires_grad=True)

# Define the function f(x) = x^2
f = x*x.conj()

g = f.real

# Perform backpropagation
g.backward()
.5* x.grad.conj()

tensor(2.-1.j)

In [73]:
def f(z):
    return z * z.conj()

x = torch.tensor(3+1j, requires_grad=True)

y = f(x).real
y.backward()
0.5*x.grad

tensor(3.+1.j)

In [74]:
def f(z):
    return (z * z.conj()).real

x = torch.tensor(3+3j, requires_grad=True)
lr = 0.1

for _ in range(100):
    # 1) zero the old gradient
    if x.grad is not None:
        x.grad.zero_()

    # 2) forward / backward
    y = f(x)
    y.backward()

    # 3) extract Wirtinger derivative
    grad = 0.5 * x.grad   # now x.grad is guaranteed not None

    # 4) gradient step in-place, no_grad so we stay a leaf
    with torch.no_grad():
        x -= lr * grad

    # x is already a leaf with requires_grad=True
    print(x)

tensor(2.7000+2.7000j, requires_grad=True)
tensor(2.4300+2.4300j, requires_grad=True)
tensor(2.1870+2.1870j, requires_grad=True)
tensor(1.9683+1.9683j, requires_grad=True)
tensor(1.7715+1.7715j, requires_grad=True)
tensor(1.5943+1.5943j, requires_grad=True)
tensor(1.4349+1.4349j, requires_grad=True)
tensor(1.2914+1.2914j, requires_grad=True)
tensor(1.1623+1.1623j, requires_grad=True)
tensor(1.0460+1.0460j, requires_grad=True)
tensor(0.9414+0.9414j, requires_grad=True)
tensor(0.8473+0.8473j, requires_grad=True)
tensor(0.7626+0.7626j, requires_grad=True)
tensor(0.6863+0.6863j, requires_grad=True)
tensor(0.6177+0.6177j, requires_grad=True)
tensor(0.5559+0.5559j, requires_grad=True)
tensor(0.5003+0.5003j, requires_grad=True)
tensor(0.4503+0.4503j, requires_grad=True)
tensor(0.4053+0.4053j, requires_grad=True)
tensor(0.3647+0.3647j, requires_grad=True)
tensor(0.3283+0.3283j, requires_grad=True)
tensor(0.2954+0.2954j, requires_grad=True)
tensor(0.2659+0.2659j, requires_grad=True)
tensor(0.23

In [70]:
x = torch.tensor(3+1j, requires_grad=True)
lr = 0.1

for _ in range(100):
    if x.grad is not None:
        x.grad.zero_()

    y = (x * x.conj()).real
    y.backward()

    # ⟵ Take the conjugate so we step along ∂f/∂z* = z
    grad = 0.5 * x.grad.conj()

    with torch.no_grad():
        x -= lr * grad

    # x is still a leaf with retains its .grad buffer
    print(x)

tensor(2.7000+1.1000j, requires_grad=True)
tensor(2.4300+1.2100j, requires_grad=True)
tensor(2.1870+1.3310j, requires_grad=True)
tensor(1.9683+1.4641j, requires_grad=True)
tensor(1.7715+1.6105j, requires_grad=True)
tensor(1.5943+1.7716j, requires_grad=True)
tensor(1.4349+1.9487j, requires_grad=True)
tensor(1.2914+2.1436j, requires_grad=True)
tensor(1.1623+2.3579j, requires_grad=True)
tensor(1.0460+2.5937j, requires_grad=True)
tensor(0.9414+2.8531j, requires_grad=True)
tensor(0.8473+3.1384j, requires_grad=True)
tensor(0.7626+3.4523j, requires_grad=True)
tensor(0.6863+3.7975j, requires_grad=True)
tensor(0.6177+4.1772j, requires_grad=True)
tensor(0.5559+4.5950j, requires_grad=True)
tensor(0.5003+5.0545j, requires_grad=True)
tensor(0.4503+5.5599j, requires_grad=True)
tensor(0.4053+6.1159j, requires_grad=True)
tensor(0.3647+6.7275j, requires_grad=True)
tensor(0.3283+7.4003j, requires_grad=True)
tensor(0.2954+8.1403j, requires_grad=True)
tensor(0.2659+8.9543j, requires_grad=True)
tensor(0.23

In [ ]:
# For fixed channel
#Setting Network Parameters
num_RX_ant = 2
num_TX_ant = 4
num_scatterers = torch.tensor([[1, 1], [1, 1]])
lam = 0.01
Ant_dist = 0.005
#Setting Locations
TX_locs = [torch.tensor([-10.0, 0.0], dtype=torch.float64), torch.tensor([0.0, 0.0], dtype=torch.float64)]
RX_locs = [torch.tensor([-10.0, -10.0], dtype=torch.float64), torch.tensor([0, -10.0], dtype=torch.float64)]
SC_locs = np.array([[torch.tensor([[-20.0, -5.0]], dtype=torch.float64), torch.tensor([[10.0, -5.0]], dtype=torch.float64)],
                     [torch.tensor([[-20.0, -5.0]], dtype=torch.float64), torch.tensor([[10.0, -5.0]], dtype=torch.float64)]])

N = Network()
N.num_RX_ant = num_RX_ant
N.num_TX_ant = num_TX_ant
N.num_scatterers = num_scatterers
N.lam = lam
N.Ant_dist = Ant_dist
N.TX_locs = TX_locs
N.RX_locs = RX_locs
N.SC_locs = SC_locs
N.rand_ph = 1

p = pair(N, 0, 1)

Br = p.calculate_Br()
Bt = p.calculate_Bt()
A = p.calculate_A()
H = Br @ A @ Bt.conj().T


Sigma0 = torch.eye(4)

